In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from ta.trend import WMAIndicator
from ta.trend import SMAIndicator
from ta.momentum import RSIIndicator
from ta.volatility import AverageTrueRange
from ta.volume import AccDistIndexIndicator

def ticker_data_processing(ticker, time_period):
    tick = yf.download(ticker, period=time_period)
    tickerData = tick
    
    #adjusting the Adj Close columns 
    tickerData = tickerData.drop(['Close'], axis =1)
    adjclose = tickerData['Adj Close']
    tickerData['Today Adj Close'] = adjclose
    tickerData['Adj Close'] = tickerData['Adj Close'].shift(-1)
    tickerData = tickerData.rename(columns={"Adj Close" : "Tomorrow Adj Close"})
    tickerData['Yesterday Adj Close'] = tickerData['Today Adj Close'].shift(+1)
    tickerData = tickerData.dropna()
    
    #using TA library to calculate and add TA data to the data set
    indicator_wma = WMAIndicator(close=tickerData["Today Adj Close"], window = 10)
    tickerData['WMA'] = indicator_wma.wma()
    
    indicator_rsi = RSIIndicator(close=tickerData["Today Adj Close"], window = 10)
    tickerData['RSI'] = indicator_rsi.rsi()
    
    indicator_adi = AccDistIndexIndicator(high=tickerData["High"], low=tickerData["Low"], close=tickerData["Today Adj Close"], volume=tickerData["Volume"])
    tickerData['ADI'] = indicator_adi.acc_dist_index()
    
    indicator_atr = AverageTrueRange(high=tickerData["High"], low=tickerData["Low"], close=tickerData["Today Adj Close"], window = 10)
    tickerData['ATR'] = indicator_atr.average_true_range()
    
    indicator_sma = SMAIndicator(close=tickerData["Today Adj Close"], window = 10)
    tickerData['SMA'] = indicator_sma.sma_indicator()
    
    #graphing the TA indicators
    fig, grph = plt.subplots(5, figsize =(20, 50))
    grph[0].plot(tickerData['WMA'])
    grph[1].plot(tickerData['RSI'])
    grph[2].plot(tickerData['ADI'])
    grph[3].plot(tickerData['ATR'])
    grph[4].plot(tickerData['SMA'])
    
    #drops N/A
    tickerData = tickerData.dropna()
    
    #splitting the test and train data and dropping volume from both
    train_set, test_set = train_test_split(tickerData, test_size=0.2, random_state=42)
    train_set = train_set.drop(['Volume'], axis=1)
    test_set = test_set.drop(['Volume'], axis=1)
    
    
    #setting up the scaler, can be swapped between MinMaxScaler and StandardScaler
    scale = ['Open', 'High', 'Low', 'Today Adj Close', 'WMA', 'RSI', 'ADI', 'ATR', 'SMA']
    scaler = StandardScaler()

    train_set_features = train_set.drop(["Tomorrow Adj Close"], axis=1)
    train_set_labels = train_set['Tomorrow Adj Close'].copy()
    
    feature_transform = scaler.fit_transform(train_set_features[scale])
    train_prepared = pd.DataFrame(columns=scale, data=feature_transform, index=train_set_features.index)
    
    test_set_features = train_set.drop(['Tomorrow Adj Close'], axis=1)
    test_set_labels = train_set['Tomorrow Adj Close'].copy()
    
    #scaling the data
    #train_prepared = scaler.fit_transform(train_set)
    test_prepared = scaler.transform(test_set_features[scale])
    
    
    return train_prepared, train_set_labels, test_prepared, test_set_labels 

ticker_data_processing('F', '5y')

In [ ]:
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_percentage_error

def svr_linear(ticker, period):
    train_prepared, train_set_labels, test_prepared, test_set_labels = ticker_data_processing(ticker, period)
    plt.close()
    svm_reg = LinearSVR(epsilon= .5, max_iter=2500)
    svm_reg.fit(train_prepared, train_set_labels)
    
    predictions = svm_reg.predict(test_prepared)
    
    scores = cross_val_score(svm_reg, train_prepared, train_set_labels, scoring="neg_mean_squared_error", cv=5 )
    
    rmse = np.sqrt(-scores)
    print("Scores:", rmse)
    print("Mean:", rmse.mean())
    print ("Standared deviation:", rmse.std())
    print("MAPE: ", mean_absolute_percentage_error(test_set_labels, predictions))
    
    
    train_prepared['Predicted Close'] = predictions.tolist()
    train_prepared['Predicted Close'].plot(label="Predicted", figsize=(24,12))
    test_set_labels.plot(label="Close")
    plt.legend()
    
svr_linear('F', '5y')

In [ ]:
from sklearn.svm import SVR

def svr_polynomial(ticker, period):
    train_prepared, train_set_labels, test_prepared, test_set_labels = ticker_data_processing(ticker, period)
    plt.close()
    svm_reg = SVR(kernel= 'poly', epsilon= .5, max_iter=2500)
    svm_reg.fit(train_prepared, train_set_labels)
    
    predictions = svm_reg.predict(test_prepared)
    
    scores = cross_val_score(svm_reg, train_prepared, train_set_labels, scoring="neg_mean_squared_error", cv=5 )
    
    rmse = np.sqrt(-scores)
    print("Scores:", rmse)
    print("Mean:", rmse.mean())
    print ("Standared deviation:", rmse.std())
    print("MAPE: ", mean_absolute_percentage_error(test_set_labels, predictions))
    
    
    train_prepared['Predicted Close'] = predictions.tolist()
    train_prepared['Predicted Close'].plot(label="Predicted", figsize=(24,12))
    test_set_labels.plot(label="Close")
    plt.legend()
    
svr_polynomial('F', '5y')

In [ ]:
def svr_rbf(ticker, period):
    train_prepared, train_set_labels, test_prepared, test_set_labels = ticker_data_processing(ticker, period)
    plt.close()
    svm_reg = SVR(kernel= 'rbf', epsilon= .5, max_iter=2500)
    svm_reg.fit(train_prepared, train_set_labels)
    
    predictions = svm_reg.predict(test_prepared)
    
    scores = cross_val_score(svm_reg, train_prepared, train_set_labels, scoring="neg_mean_squared_error", cv=5 )
    
    rmse = np.sqrt(-scores)
    print("Scores:", rmse)
    print("Mean:", rmse.mean())
    print ("Standared deviation:", rmse.std())
    print("MAPE: ", mean_absolute_percentage_error(test_set_labels, predictions))
    
    
    train_prepared['Predicted Close'] = predictions.tolist()
    train_prepared['Predicted Close'].plot(label="Predicted", figsize=(24,12))
    test_set_labels.plot(label="Close")
    plt.legend()
    
svr_rbf('F', '5y')